# 从excel中提出需要的那一列
<img src="data/示例图片.png" style="width:800;height:600px;"> <br>
<caption><center>部分实验数据 </center></caption>

# 整理得到所有数据
| 序号 | 最高学历 | 继续教育 | 辞职频率 | 执业资格 | 工作年限 | 工作单位 | 工作业绩 | 同行评价 | 技术水平 |   业主满意度 | 第三方投诉情况 | 司法记录 | 纳税记录 | 信贷记录 | 资质注册记录 | 获奖记录 | 执业不良行为记录 |
|----|------|------|------|------|------|------|------|------|------|---------|---------|------|------|------|--------|------|----------|
| 1  | 4    | 4    | 4    | 3    | 4    | 4    | 5    | 4    | 4    | 4       | 4       | 3    | 3    | 3    | 4      | 4    | 5        |
| 2  | 3    | 3    | 5    | 4    | 4    | 3    | 5    | 4    | 5    | 4       | 4       | 3    | 3    | 3    | 5      | 5    | 5        |
| 3  | 3    | 4    | 4    | 5    | 4    | 4    | 5    | 4    | 5    | 4       | 4       | 3    | 3    | 3    | 5      | 5    | 5        |

$$整理出的数据像这样$$

# 数据标准化
标准化之后的数据用$Y_ij$表示，i为行坐标，取值1~n，表示数据个数；j为列坐标，取值1~k，表示指标个数

公式不再详述，实现为sklearn中的MinMaxScaler
# 计算信息熵的公式为：$E_j=-ln(n)^{-1}\sum_{i=1}^{n}[p_{ij}{lnp_{ij}}]$

其中**$p_{ij}=Y_{ij}/\sum_{i=1}^{n}Y_{ij}$**

整个矩阵可以用$Y_{ij}$表示，即可以直接用data/**$\sum_{i=1}^{n}Y_{ij}$**

中括号的部分以下用value指代

# 计算各指标的权重：$W_j = \frac{1-E_j}{k-\sum{E_j}}$
# 对每条输入数据进行评分：$S_i=\sum_{j=1}^{k}X_{ij}W_j$
$S_i$为第$i$条输入的在$k$个指标共同作用下的最终评分，比如有n条数据，输入矩阵shape=（n,k），权重矩阵shape为（k,1)

矩阵相乘正好(n,k)*(k,1)=(n,1),这样便会得到最终的得分矩阵


In [92]:
import pandas as pd
import numpy as np
import os
import openpyxl
from sklearn.preprocessing import MinMaxScaler

#1. 从excel中提出需要的那一列
def get_data(xlsx):
    
    '''
    --输入：.xlsx文件
    --输出：单个问卷打分列表，提取得到的文件中标黄的数据
    
    '''
    
    #workbook打开excel，worksheet为当前sheet
    workbook=openpyxl.load_workbook(xlsx)
    worksheet=workbook.get_active_sheet()
    #excel最大行列
    rows,cols=worksheet.max_row,worksheet.max_column
    #把excel中的标黄数据提出来
    yellow=[]
    #excel的索引值从1开始
    for i in range(1,rows+1):
        for j in range(1,cols+1):
            ce = worksheet.cell(row=i,column=j)
            fill = ce.fill
            font = ce.font
            #黄色在excel中的RGB代码是"FFFFFF00",前两个FF应该表示透明度
            if fill.start_color.rgb=="FFFFFF00":
                yellow.append(ce.value)
    
    single_score = []
    for i in yellow:
        #以防有的表格是有打勾的情况，把str类型取第一位数值加入到列表中
        if isinstance(i,str):
            one_choice = int(i[0:1])
        else:
            one_choice = int(i)
        #把每个选项加到列表，最后得到单个问卷的数据
        single_score.append(one_choice)
    return single_score

#2.整理得到所有数据
def get_all_survey_data(excel_path): 
    
    '''
    --输入：所有excel表的存储位置，假设的情况是一个问卷一个excel
    --输出：收集到的所有问卷调查结果,numpy.array格式
    
    '''
    ques_table = []
    #读取文件夹中所有excel，如果每个问卷单独成一个excel的话
    all_excel = os.listdir(excel_path)
    for i in all_excel:
        single_ques = get_data(excel_path+'//'+i)
        ques_table.append(single_ques)
    #转成numpy.array格式以便之后的矩阵操作
    my_data = np.array(ques_table)
    return my_data


def get_entropy_weight_and_score(my_data):
    #3.数据标准化，实现为sklearn中的MinMaxScaler
    scaler = MinMaxScaler()
    standard = scaler.fit_transform(my_data)
    n,k = standard.shape   #n是行，k是列
    #axis = 0表示按行执行函数
    Y_ij = standard.sum(axis=0)
    #计算pij
    p_ij = standard/Y_ij
    #把nan全部改成0
    value = np.nan_to_num(p_ij*np.log(p_ij))
    #4.计算出各指标的信息熵
    E_j = -(1/np.log(n))*(value.sum(axis=0))
    #5.计算各指标的权重
    W_j = (1-E_j)/(k-np.sum(E_j))
    #6.最后直接一波矩阵乘法，权重乘以标准化之前的值，得到分数
    S_i = np.matmul(my_data,W_j)
    
    return E_j,W_j,S_i

if __name__ == '__main__':
    #需要修改的地方
    excel_path = './data'
    my_data = get_all_survey_data(excel_path)
    E,W,S = get_entropy_weight_and_score(my_data)
    print('原始输入：',my_data)
    print('======================================================================')
    print('信息熵：',E)
    print('======================================================================')
    print('各指标权重：',W)
    print('======================================================================')
    print('各问卷得分：',S)

原始输入： [[4 4 4 3 4 4 5 4 4 4 4 3 3 3 4 4 5]
 [4 4 4 3 4 4 5 4 4 4 4 3 3 3 4 4 5]
 [4 4 4 3 4 4 5 4 4 4 4 3 3 3 4 4 5]]
信息熵： [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
各指标权重： [0.05882353 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353
 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353
 0.05882353 0.05882353 0.05882353 0.05882353 0.05882353]
各问卷得分： [3.88235294 3.88235294 3.88235294]
